In [2]:
pip install jovian

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/68.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.6 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/68.6 kB ? eta -:--:--
   ----------------- ---------------------- 30.7/68.6 kB 660.6 kB/s eta 0:00:01
   ----------------------- ---------------- 41.0/68.6 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------- 68.6/68.6 kB 535.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/97.9 kB ? eta -:--:--
   ---------------- ----------------------- 41.0/97.9 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------- 97.9/97.9 kB 943.0 kB/s eta 0:00:00
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6483 sha256=16785cab6a328b62d6aec664f85443bbbb35349b56a2df52b05d48f38b18750e
  

In [5]:
import jovian
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

<IPython.core.display.Javascript object>

In [4]:
URL = 'https://books.toscrape.com/'
response = requests.get(URL)

page_contents = response.text

#Creating a file and loading the page contents in it.
with open('Bookswebpage.html','w') as f:
    f.write(page_contents)

In [5]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents,'html.parser')

In [6]:
def get_book_titles(doc):
    Book_title_tags = doc.find_all('h3')
    Book_titles = []
    for tags in Book_title_tags:
        Book_titles.append(tags.text)
    return Book_titles

In [7]:
get_book_titles(doc)

['A Light in the ...',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History ...',
 'The Requiem Red',
 'The Dirty Little Secrets ...',
 'The Coming Woman: A ...',
 'The Boys in the ...',
 'The Black Maria',
 'Starving Hearts (Triangular Trade ...',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little ...",
 'Rip it Up and ...',
 'Our Band Could Be ...',
 'Olio',
 'Mesaerion: The Best Science ...',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

In [8]:
def get_book_price(doc):
    Book_price_tags = doc.find_all('p', class_ = 'price_color')
    Book_price = []
    for tags in Book_price_tags:
        Book_price.append(tags.text.replace('Â',''))
    return Book_price

In [9]:
get_book_price(doc)

['£51.77',
 '£53.74',
 '£50.10',
 '£47.82',
 '£54.23',
 '£22.65',
 '£33.34',
 '£17.93',
 '£22.60',
 '£52.15',
 '£13.99',
 '£20.66',
 '£17.46',
 '£52.29',
 '£35.02',
 '£57.25',
 '£23.88',
 '£37.59',
 '£51.33',
 '£45.17']

In [10]:
def get_stock_availability(doc):
    Book_stock_tags = doc.find_all('p', class_ = 'instock availability')
    Book_stock = []
    for tags in Book_stock_tags:
        Book_stock.append(tags.text.strip())
    return Book_stock

In [6]:
url = 'https://books.toscrape.com/'
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    Book_title_tags = soup.find_all('h3')
else:
    print('Gagal mengambil halaman web')

In [7]:
def get_book_url(Book_title_tags):
    Book_url = []
    for article in Book_title_tags:
        for link in article.find_all('a', href = True):
            url = link['href']
            links = 'https://books.toscrape.com/' + url
            if links not in Book_url:
                Book_url.append(links)
    return Book_url

In [13]:
get_stock_availability(doc)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

In [8]:
get_book_url(Book_title_tags)

['https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'https://books.toscrape.com/catalogue/soumission_998/index.html',
 'https://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'https://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
 'https://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'https://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'https://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'https://books.toscrape.com/catalogue/the-black-maria_991/index.html',
 'https://books.toscrape.com/catalogue/starving-hearts-triangular-trade-tr

In [15]:
def get_doc(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.text,'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(response))
    return doc

In [16]:
def scrape_multiple_pages(n):
    URL = 'https://books.toscrape.com/catalogue/page-'
    titles,prices,stocks_availability,urls = [],[],[],[]
    
    for page in range(1,n+1):
        doc = get_doc(URL + str(page)+ '.html')
        titles.extend(get_book_titles(doc))
        prices.extend(get_book_price(doc))
        stocks_availability.extend(get_stock_availability(doc))
        urls.extend(get_book_url(doc.find_all('h3')))
        
    book_dict1 = {
                'TITLE':titles,
                'PRICE':prices,
                'STOCK AVAILABILTY':stocks_availability,
                'URL':urls}
    return pd.DataFrame(book_dict1)

In [17]:
scrape_multiple_pages(5)

,TITLE,PRICE,STOCK AVAILABILTY,URL
0,A Light in the ...,£51.77,In stock,https://books.toscrape.com/a-light-in-the-atti...
1,Tipping the Velvet,£53.74,In stock,https://books.toscrape.com/tipping-the-velvet_...
2,Soumission,£50.10,In stock,https://books.toscrape.com/soumission_998/inde...
3,Sharp Objects,£47.82,In stock,https://books.toscrape.com/sharp-objects_997/i...
4,Sapiens: A Brief History ...,£54.23,In stock,https://books.toscrape.com/sapiens-a-brief-his...
...,...,...,...,...
95,Lumberjanes Vol. 3: A ...,£19.92,In stock,https://books.toscrape.com/lumberjanes-vol-3-a...
96,"Layered: Baking, Building, and ...",£40.11,In stock,https://books.toscrape.com/layered-baking-buil...
97,Judo: Seven Steps to ...,£53.90,In stock,https://books.toscrape.com/judo-seven-steps-to...
98,Join,£35.67,In stock,https://books.toscrape.com/join_902/index.html


In [18]:
scrape_multiple_pages(5).to_csv('SCB.csv',index = None)